# Extracción de datos de la base de datos de PTB Diagnostic ECG Database

In [1]:
import wfdb
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, find_peaks

In [2]:
folder_name = './data/ptb-diagnostic-ecg-database-1.0.0'
files = []
diagnosis_per_patient = {}
kinds_of_diagnosis = set()
patients_per_diagnosis = {}

In [3]:
def bandpass_filter(signal, lowcut, highcut, fs, order=1):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, signal)

In [4]:
def calculate_hr(p_record_name):
    record = wfdb.rdrecord(p_record_name)
    ecg_data = record.p_signal  # Load all 12-lead ECG signals

    # Step 2: Select Lead II for Heart Rate Calculation
    lead_II = ecg_data[:, 1]  # Assuming lead II is the second column
    
    fs = 1000  # Sampling frequency is 1000 Hz
    filtered_lead_II = bandpass_filter(lead_II, 0.5, 50, fs)
    
    # Step 4: R-Peak Detection
    # Use the find_peaks function to detect R-peaks
    peaks, _ = find_peaks(filtered_lead_II, distance=fs*0.6)  # Assuming a minimum distance of 600ms between peaks
    
    # Step 5: Calculate RR Intervals and Heart Rate
    rr_intervals = np.diff(peaks) / fs  # RR intervals in seconds
    hr_values = 60 / rr_intervals  # Heart rate in beats per minute (bpm)
    
    # Step 6: Generate Time Axis for Heart Rate Plot
    time_peaks = peaks / fs  # Time of R-peaks in seconds
    time_hr = (time_peaks[:-1] + time_peaks[1:]) / 2  # Midpoint between successive peaks
    return time_hr, hr_values

In [5]:
def plot_hr_signal(time_hr, hr_values):
    plt.figure(figsize=(10, 6))
    plt.plot(time_hr, hr_values, label='Heart Rate (bpm)', color='b', marker='o', linestyle='-')
    plt.title('Heart Rate over Time')
    plt.xlabel('Time (s)')
    plt.ylabel('Heart Rate (bpm)')
    plt.grid(True)
    plt.legend()
    plt.show()

In [6]:
# Descomentar para visualizar métricas de diagnóstico
# with open(folder_name + '/RECORDS') as f:
#     for line in f:
#         record_name = line.strip()
#         file_name = f'{folder_name}/{record_name}'
#         files.append(file_name)
#         header = wfdb.rdheader(file_name)
#         diagnosis = header.comments[4].split(': ')[1]
#         diagnosis_per_patient[f'{record_name}'] = diagnosis
#         kinds_of_diagnosis.add(diagnosis)
#         if diagnosis in patients_per_diagnosis:
#             patients_per_diagnosis[diagnosis].append(record_name)
#         else:
#             patients_per_diagnosis[diagnosis] = [record_name]
#         time_hr, hr_values = calculate_hr(file_name)
#         plot_hr_signal(time_hr, hr_values)



In [7]:
def calcular_min_hrv(hr):
    min_hrv = 1000
    previous_hr = 0
    for i in range(len(hr)):
        if i == 0:
            previous_hr = hr[i]
        else:
            hrv = abs(hr[i] - previous_hr)
            if hrv != 0 and hrv < min_hrv:
                min_hrv = hrv
            previous_hr = hr[i]
    return min_hrv

In [8]:
def calcular_max_hrv(hr):
    max_hrv = 0
    previous_hr = 0
    for i in range(len(hr)):
        if i == 0:
            previous_hr = hr[i]
        else:
            hrv = abs(hr[i] - previous_hr)
            if hrv > max_hrv:
                max_hrv = hrv
            previous_hr = hr[i]
    return max_hrv

In [9]:
def calcular_mean_hrv(hr):
    mean_hrv = 0
    previous_hr = 0
    for i in range(len(hr)):
        if i == 0:
            previous_hr = hr[i]
        else:
            hrv = abs(hr[i] - previous_hr)
            mean_hrv += hrv
            previous_hr = hr[i]
    return mean_hrv / len(hr)

In [10]:
def calcular_median_hrv(hr):
    hrvs = []
    previous_hr = 0
    for i in range(len(hr)):
        if i == 0:
            previous_hr = hr[i]
        else:
            hrv = abs(hr[i] - previous_hr)
            hrvs.append(hrv)
            previous_hr = hr[i]
    return np.median(hrvs)

In [11]:
def calcular_std_hrv(hr):
    hrvs = []
    previous_hr = 0
    for i in range(len(hr)):
        if i == 0:
            previous_hr = hr[i]
        else:
            hrv = abs(hr[i] - previous_hr)
            hrvs.append(hrv)
            previous_hr = hr[i]
    return np.std(hrvs)

In [12]:
def count_outliers(hr):
    q1 = np.percentile(hr, 5)
    q3 = np.percentile(hr, 95)
    outliers = 0
    for value in hr:
        if value < q1 or value > q3:
            outliers += 1
    return outliers

In [13]:
import pandas as pd
from scipy.signal import find_peaks
from hr_engine import get_features


times = pd.DataFrame()
hear_rates = pd.DataFrame()
process_only_one = False
results = []
with open(folder_name + '/RECORDS') as f:
    for line in f:
        
        record_name = line.strip()
        file_name = f'{folder_name}/{record_name}'
        files.append(file_name)
        header = wfdb.rdheader(file_name)
        diagnosis = header.comments[4].split(': ')[1]
        if diagnosis == 'n/a':
            continue
        diagnosis_per_patient[f'{record_name}'] = diagnosis
        kinds_of_diagnosis.add(diagnosis)
        if diagnosis in patients_per_diagnosis:
            patients_per_diagnosis[diagnosis].append(record_name)
        else:
            patients_per_diagnosis[diagnosis] = [record_name]
        time_hr, hr_values = calculate_hr(file_name)
        
        # remove first 5 and last 5 values of time_hr and hr_values
        time_hr = time_hr[5:-5]
        hr_values = hr_values[5:-5]

        if diagnosis == 'n/a':
            continue
        
        diagnosis = 0 if diagnosis == "Healthy control" else 1
        
        results.append(get_features(hr_values=hr_values, time_hr=time_hr, record_name=record_name, diagnosis=diagnosis))

        if process_only_one:
            break
print(kinds_of_diagnosis)
df = pd.DataFrame(results)
df.head()

# print(kinds_of_diagnosis)
        

/home/gino/Documents/Repos/pia-02/hr_engine.py:137: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hr_smoothed = pd.Series(hr_values).rolling(window=window_size, center=True).mean().fillna(method='bfill').fillna(
/home/gino/Documents/Repos/pia-02/hr_engine.py:137: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hr_smoothed = pd.Series(hr_values).rolling(window=window_size, center=True).mean().fillna(method='bfill').fillna(
/home/gino/Documents/Repos/pia-02/hr_engine.py:137: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hr_smoothed = pd.Series(hr_values).rolling(window=window_size, center=True).mean().fillna(method='bfill').fillna(
/home/gino/Documents/Repos/pia-02/hr_engine.py:137: FutureWarning: Series.fillna with 'method' is

{'Myocarditis', 'Healthy control', 'Heart failure (NYHA 2)', 'Unstable angina', 'Palpitation', 'Valvular heart disease', 'Heart failure (NYHA 3)', 'Cardiomyopathy', 'Stable angina', 'Myocardial infarction', 'Dysrhythmia', 'Hypertrophy', 'Heart failure (NYHA 4)', 'Bundle branch block'}


,patient,diagnosis,highest_heart_rate,lowest_heart_rate,mean_heart_rate,median_heart_rate,standard_deviation_hr,minimum_hrv,maximum_hrv,mean_hrv,...,hf_power,mean_rr,standard_deviation_rr,minimum_rr,maximum_rr,mean_deviation,tendency_standard_deviation,approximation_entropy,sample_entropy,outliers_percentage
0,patient001/s0010_re,1,85.106383,79.155673,81.959865,81.743869,1.188020,0.110015,3.831879,1.310418,...,43.056090,2.484850,0.806827,1.4595,4.3460,-2.079637e-15,0.211825,0.181738,1.791759,0.073171
1,patient001/s0014lre,1,93.023256,54.005401,85.991094,86.206897,3.855155,0.115902,33.458156,3.368700,...,819.455368,1.990817,0.590739,1.3675,3.4795,1.254985e-14,1.494654,1.186733,1.996060,0.071429
2,patient001/s0016lre,1,99.502488,49.751244,79.181158,79.681275,5.319190,0.101858,32.215969,3.810716,...,1893.184563,2.271467,0.774889,1.4635,4.4645,-5.845600e-15,1.782076,1.174023,1.787364,0.085106
3,patient002/s0015lre,1,85.714286,75.376884,78.832471,78.534031,2.099841,0.094814,2.659452,0.710673,...,250.711688,3.439446,1.575644,1.5375,7.5300,7.458179e-15,1.623624,0.880340,1.575536,0.078014
4,patient003/s0017lre,1,78.843627,68.259386,72.259853,72.115385,2.039696,0.082173,1.821804,0.454302,...,123.343219,6.391933,2.915702,1.6135,10.9075,-3.330669e-16,1.779684,0.679967,1.006442,0.109375


In [14]:
df.to_csv('ptb-diagnostic-clear-data.csv', index=False)